In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## DAY 1 MACHINE LEARNING WITH PYTHON/DAY1 MACHINE LEARNING IN PYTHON ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [ ]:
#=================================================-
#### Slide 5: Loading the packages  ####

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.vq import kmeans
from scipy.spatial.distance import cdist,pdist
from matplotlib import cm




In [ ]:
#=================================================-
#### Slide 6: Directory settings  ####

# Set 'main_dir' to location of the project folder
from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




In [ ]:
#=================================================-
#### Slide 16: Data prep  ####

temp_heart = pd.read_csv("temp_heart_rate.csv")
print(temp_heart.head())




In [ ]:
#=================================================-
#### Slide 17: Data cleaning: NAs  ####

print(temp_heart.isnull().sum())




In [ ]:
#=================================================-
#### Slide 18: Subset data  ####

# Subset even further to just have 'Body Temp' and 'Heart Rate'.
temp_heart_cluster = temp_heart[['Body Temp','Heart Rate']]
print(temp_heart_cluster.head())




In [ ]:
#=================================================-
#### Slide 19: Numeric variables  ####

# Check data type of our variables.
print(temp_heart_cluster.dtypes)




In [ ]:
#=================================================-
#### Slide 22: K-means data prep using  MinMaxScaler  ####

# Instantiate MinMaxScaler.
scaler = MinMaxScaler()

# Scale the dataframe.
temp_heart_cluster_scaled = scaler.fit_transform(temp_heart_cluster)
# Convert back to dataframe, making sure to name the columns again.
temp_heart_kmeans = pd.DataFrame(temp_heart_cluster_scaled, columns = temp_heart_cluster.columns)
print(temp_heart_kmeans.head())




In [ ]:
#=================================================-
#### Slide 28: K-means: Body Temp vs Heart Rate  ####

# Plot the data.
plt.scatter(temp_heart_kmeans['Body Temp'],
            temp_heart_kmeans['Heart Rate'])

plt.title('Body temp vs Heart Rate')
plt.ylabel('Heart Rate')
plt.xlabel('Body Temp')





In [ ]:
#=================================================-
#### Slide 29: K-means: k-means with k=2  ####

# K-means - start with 2 clusters.
# Initializing k-means.
kmeans_2 = KMeans(n_clusters=2)
# Fitting with inputs.
kmeans_2 = kmeans_2.fit(temp_heart_kmeans)
# Predicting the clusters.
labels = kmeans_2.predict(temp_heart_kmeans)
# Getting the cluster centers.
C_2 = kmeans_2.cluster_centers_
print(C_2)




In [ ]:
#=================================================-
#### Slide 30: K-means: plot k=2  ####

# First, we plot our clusters, colored in by the labels.
plt.scatter(temp_heart_kmeans.iloc[:,0],
            temp_heart_kmeans.iloc[:,1],
            c=kmeans_2.labels_,
            cmap='rainbow')
# Second, we plot the optimized centroids over the clusters.
plt.scatter(C_2[:, 0],
            C_2[:, 1],
            c='black',
            s=200,
            alpha=0.5)




In [ ]:
#=================================================-
#### Slide 32: Exercise 1  ####






In [ ]:
#=================================================-
#### Slide 40: Elbow method  ####

# Set the range of k.
K_MAX = 20
KK = range(1,K_MAX+1)
# Run `kmeans` for values in the range k = 1-20.
KM = [kmeans(temp_heart_kmeans,k) for k in KK]
# Find the centroids for each KM output. 
centroids = [cent for (cent,var) in KM]
# Calculate centroids for each iteration of k. 
D_k = [cdist(temp_heart_kmeans, cent, 'euclidean') for cent in centroids]
cIdx = [np.argmin(D,axis=1) for D in D_k]
dist = [np.min(D,axis=1) for D in D_k]
tot_withinss = [sum(d**2) for d in dist]                        # Total within-cluster sum of squares
totss = sum(pdist(temp_heart_kmeans)**2)/temp_heart_kmeans.shape[0]   # The total sum of squares
betweenss = totss - tot_withinss                                # The between-cluster sum of squares




In [ ]:
#=================================================-
#### Slide 42: K-means: Elbow plot method  ####

# Set range for k.
kIdx = 4        # K=5
clr = cm.Spectral( np.linspace(0,1,10) ).tolist()
mrk = 'os^p<dvh8>+x.'

# Elbow curve - explained variance.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(KK, betweenss/totss*100, 'b*-')
ax.plot(KK[kIdx], betweenss[kIdx]/totss*100, 
        marker='o', markersize=12, 
        markeredgewidth=2, markeredgecolor='r', 
        markerfacecolor='None')
ax.set_ylim((0,100))
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Percentage of variance explained (%)')
plt.title('Elbow for KMeans clustering')




In [ ]:
#=================================================-
#### Slide 44: K-means: silhouette method  ####

obs = temp_heart_kmeans
silhouette_score_values=list()
NumberOfClusters = range(2,30)
for i in NumberOfClusters:
    classifier=cluster.KMeans(i,init='k-means++', n_init=10, 
                              max_iter=300, 
                              tol=0.0001, 
                              verbose=0, 
                              random_state=None, 
                              copy_x=True)
    classifier.fit(obs)
    labels= classifier.predict(obs)
    sklearn.metrics.silhouette_score(obs,labels ,metric='euclidean', sample_size=None, random_state=None)
    silhouette_score_values.append(sklearn.metrics.silhouette_score(obs,labels ,metric='euclidean', sample_size=None, random_state=None))
plt.plot(NumberOfClusters, silhouette_score_values)
plt.title("Silhouette score values vs Numbers of Clusters ")




In [ ]:
#=================================================-
#### Slide 45: K-means: silhouette method  ####

Optimal_NumberOf_Components=NumberOfClusters[silhouette_score_values.index(max(silhouette_score_values))]
print("Optimal number of components is:", Optimal_NumberOf_Components)




In [ ]:
#=================================================-
#### Slide 47: Baseline vs. optimal k  ####

# Explained variance for optimal number of clusters at `k = 2`.
print(betweenss[1]/totss * 100)
# Explained variance for optimal number of clusters at `k = 5`.
print(betweenss[4]/totss * 100)




In [ ]:
#=================================================-
#### Slide 48: Run optimal k  ####

# Initializing K-means.
kmeans_5 = KMeans(n_clusters = 5)
# Fitting with inputs.
kmeans_5 = kmeans_5.fit(temp_heart_kmeans)
# Predicting the clusters.
labels = kmeans_5.predict(temp_heart_kmeans)
# Getting the cluster centers.
C_5 = kmeans_5.cluster_centers_




In [ ]:
#=================================================-
#### Slide 49: Plot k = 5  ####

# First we plot our clusters, colored in by the labels.
plt.scatter(temp_heart_kmeans.iloc[:,0],            
            temp_heart_kmeans.iloc[:,1], 
            c = kmeans_5.labels_, 
            cmap = 'rainbow')
# Second, we plot the optimized centroids over the clusters.
plt.scatter(C_5[:, 0], 
            C_5[:, 1], 
            c = 'black', 
            s = 200, 
            alpha = 0.5)




In [ ]:
#=================================================-
#### Slide 50: Plot k = 2 vs. k = 5  ####

plt.clf()
plt.rcParams.update({'font.size': 20})
plt.figure(figsize = (16, 8))

plt.subplot(1, 2, 1)
plt.scatter(temp_heart_kmeans.iloc[:,0],temp_heart_kmeans.iloc[:,1],c=kmeans_2.labels_,cmap='rainbow')
plt.scatter(C_2[:, 0], C_2[:, 1], c='black', s=200,alpha=0.5)

plt.subplot(1, 2, 2)
plt.scatter(temp_heart_kmeans.iloc[:,0],temp_heart_kmeans.iloc[:,1],
c=kmeans_5.labels_, cmap='rainbow')
plt.scatter(C_5[:, 0], C_5[:, 1], c='black', s=200, alpha=0.5)





In [ ]:
#=================================================-
#### Slide 52: Inspect clusters  ####

# Append the other variables back to the dataframe with clusters.
clustered_temp_heart = temp_heart_cluster 
# Add cluster numbers.
clustered_temp_heart['clusters'] = pd.Series(labels)
clustered_temp_heart.head()




In [ ]:
#=================================================-
#### Slide 53: Inspect clusters  ####

# Group by `clusters` column to see the group mean of each variable.
cluster_groups_means = clustered_temp_heart.groupby('clusters').mean()
print(cluster_groups_means)


